<a href="https://colab.research.google.com/github/mifm/mifm_test/blob/master/battery_power_zero_bug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git@Megha_P2X_BM_v1_4

  Cloning https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git (to revision Megha_P2X_BM_v1_4) to /tmp/pip-req-build-j7p_1v32
  Running command git clone --filter=blob:none --quiet https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git /tmp/pip-req-build-j7p_1v32
  Running command git checkout -b Megha_P2X_BM_v1_4 --track origin/Megha_P2X_BM_v1_4
  Switched to a new branch 'Megha_P2X_BM_v1_4'
  Branch 'Megha_P2X_BM_v1_4' set up to track remote branch 'Megha_P2X_BM_v1_4' from 'origin'.
  Resolved https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git to commit 436440f870f2f0d63ff1c3f5a9fcd92e61d4450a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 80.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.2/819.2 

In [2]:
import os
import time
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from hydesign.assembly.hpp_assembly_P2X import hpp_model_P2X as hpp_model
from hydesign.examples import examples_filepath

In [3]:
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0, sep=';')
examples_sites

,case,name,longitude,latitude,altitude,input_ts_fn,sim_pars_fn,price_fn,price_col,H2_demand_col,Unnamed: 11,input_HA_ts_fn,price_up_ts,price_dwn_ts
0,India,Indian_site_good_wind,77.500226,8.334294,679.803454,India/GWA2/input_ts_Indian_site_good_wind.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price,India/H2_demand.csv,NaN,NaN,NaN,NaN
1,India,Indian_site_good_solar,68.542204,23.542099,29.883557,India/GWA2/input_ts_Indian_site_good_solar.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price,India/H2_demand.csv,NaN,NaN,NaN,NaN
2,India,Indian_site_bad_solar_bad_wind,77.916878,17.292316,627.424643,India/GWA2/input_ts_Indian_site_bad_solar_bad_...,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price,India/H2_demand.csv,NaN,NaN,NaN,NaN
3,Europe,France_good_solar,4.229736,44.422011,204.000000,Europe/GWA2/input_ts_France_good_solar.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R,Europe/H2_demand.csv,NaN,NaN,NaN,NaN
4,Europe,France_good_wind,-0.864258,48.744116,302.000000,Europe/GWA2/input_ts_France_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R,Europe/H2_demand.csv,NaN,NaN,NaN,NaN
5,Europe,France_bad_solar_n_wind,2.167969,47.428087,140.000000,Europe/GWA2/input_ts_France_bad_solar_n_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R,Europe/H2_demand.csv,NaN,NaN,NaN,NaN
6,Europe,Germany_bad_solar_n_wind,10.766602,49.310798,442.000000,Europe/GWA2/input_ts_Germany_bad_solar_n_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DE_ME,Europe/H2_demand.csv,NaN,NaN,NaN,NaN
7,Europe,Germany_good_wind,7.873535,53.287111,5.000000,Europe/GWA2/input_ts_Germany_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DE_NW,Europe/H2_demand.csv,NaN,NaN,NaN,NaN
8,Europe,Denmark_good_solar,11.813965,55.397760,42.000000,Europe/GWA2/input_ts_Denmark_good_solar.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DK_E,Europe/H2_demand.csv,NaN,NaN,NaN,NaN
9,Europe,Denmark_good_wind,8.594398,56.227322,85.000000,Europe/GWA2/input_ts_Denmark_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DK_W,Europe/H2_demand.csv,NaN,NaN,NaN,NaN


In [10]:
name = 'France_good_wind'

ex_site = examples_sites.loc[examples_sites.name == name]

longitude = ex_site['longitude'].values[0]
latitude = ex_site['latitude'].values[0]
altitude = ex_site['altitude'].values[0]

In [11]:
# Weather data and Price data
input_ts_fn = examples_filepath+ex_site['input_ts_fn'].values[0]

input_ts = pd.read_csv(input_ts_fn, index_col=0, parse_dates=True)

required_cols = [col for col in input_ts.columns if 'WD' not in col]
input_ts = input_ts.loc[:,required_cols]
input_ts

,WS_1,WS_50,WS_100,WS_150,WS_200,temp_air_1,ghi,dni,dhi,Price
2012-01-01 00:00:00,3.816262,9.189450,11.084209,12.271178,13.193283,284.873662,0.10670,0.0,0.10670,32.685
2012-01-01 01:00:00,3.433942,8.425198,10.193588,11.295483,12.177159,284.759079,0.05335,0.0,0.05335,32.685
2012-01-01 02:00:00,3.401840,8.457382,10.257751,11.500057,12.475967,284.682409,0.00000,0.0,0.00000,32.685
2012-01-01 03:00:00,3.552851,8.750878,10.595812,11.846432,12.862474,284.804835,0.00000,0.0,0.00000,32.685
2012-01-01 04:00:00,3.563602,8.849321,10.730704,12.023872,13.075707,284.912742,0.00000,0.0,0.00000,32.685
...,...,...,...,...,...,...,...,...,...,...
2012-12-30 19:00:00,3.431328,8.738007,10.644316,11.514788,12.815824,281.205509,0.00000,0.0,0.00000,37.591
2012-12-30 20:00:00,3.522005,8.956219,10.906332,11.806788,13.153907,281.139846,0.00000,0.0,0.00000,35.839
2012-12-30 21:00:00,3.546788,9.081438,11.071820,12.064809,13.412334,280.716650,0.00000,0.0,0.00000,35.362
2012-12-30 22:00:00,3.613134,9.352224,11.424145,12.542857,13.966094,280.646177,0.00000,0.0,0.00000,34.468


In [12]:
# Hydrogen demand data, when H2 offtake is infinite -> make H2_demand values very high (1e6) in H2_demand.csv file and penalty_H2 as '0' in hpp_pars.yml file
H2_demand_fn = examples_filepath+ex_site['H2_demand_col'].values[0]

H2_demand_ts = pd.read_csv(H2_demand_fn, index_col=0, parse_dates=True)
H2_demand_ts

,H2_demand
2012-01-01 00:00:00,10000
2012-01-01 01:00:00,10000
2012-01-01 02:00:00,10000
2012-01-01 03:00:00,10000
2012-01-01 04:00:00,10000
...,...
2012-12-31 19:00:00,10000
2012-12-31 20:00:00,10000
2012-12-31 21:00:00,10000
2012-12-31 22:00:00,10000


In [13]:
# Input data of technology's cost
sim_pars_fn = examples_filepath+ex_site['sim_pars_fn'].values[0]

with open(sim_pars_fn) as file:
    sim_pars = yaml.load(file, Loader=yaml.FullLoader)

print(sim_pars_fn)
sim_pars

/usr/local/lib/python3.10/dist-packages/hydesign/examples/Europe/hpp_pars.yml


{'G_MW': 300,
 'year': '2012',
 'N_life': 25,
 'wind_turbine_cost': 640000,
 'wind_civil_works_cost': 260000,
 'wind_fixed_onm_cost': 12600,
 'wind_variable_onm_cost': 1.35,
 'd_ref': 145,
 'hh_ref': 100,
 'p_rated_ref': 5.0,
 'wpp_efficiency': 1,
 'wind_deg_yr': [0, 25],
 'wind_deg': [0, 0.25],
 'share_WT_deg_types': 0.5,
 'solar_PV_cost': 110000,
 'solar_hardware_installation_cost': 100000,
 'solar_inverter_cost': 20000,
 'solar_fixed_onm_cost': 4500,
 'land_use_per_solar_MW': 0.01226,
 'tracking': 'No',
 'pv_deg_yr': [0, 1, 25],
 'pv_deg': [0.03, 0.03, 0.28],
 'battery_energy_cost': 62000,
 'battery_power_cost': 16000,
 'battery_BOP_installation_commissioning_cost': 80000,
 'battery_control_system_cost': 2250,
 'battery_energy_onm_cost': 0,
 'battery_depth_of_discharge': 0.9,
 'battery_charge_efficiency': 0.985,
 'battery_price_reduction_per_year': 0.05,
 'min_LoH': 0.7,
 'n_full_power_hours_expected_per_day_at_peak_price': 0,
 'peak_hr_quantile': 0.9,
 'price_H2': 5,
 'storage_eff'

In [14]:
hpp = hpp_model(
        latitude,
        longitude,
        altitude,
        num_batteries = 3,
        work_dir = './',
        sim_pars_fn = sim_pars_fn,
        input_ts_fn = input_ts_fn,
        H2_demand_fn = H2_demand_fn,
)


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0


In [21]:
start = time.time()

clearance = 10
sp = 360
p_rated = 4
Nwt = 90
wind_MW_per_km2 = 5
solar_MW = 80
surface_tilt = 50
surface_azimuth = 210
DC_AC_ratio = 1.5
b_P = 0
b_E_h  = 4
cost_of_batt_degr = 10
ptg_MW = 150
HSS_kg = 3000

x = [clearance, sp, p_rated, Nwt, wind_MW_per_km2, solar_MW, surface_tilt, surface_azimuth, DC_AC_ratio, b_P, b_E_h, cost_of_batt_degr, ptg_MW, HSS_kg]

outs = hpp.evaluate(*x)

hpp.print_design(x, outs)

end = time.time()
print(f'exec. time [min]:', (end - start)/60 )

AttributeError: 'ems' <class ems_P2X>: Error calling compute(), 'NoneType' object has no attribute 'get_value_dict'